# extract, transform, load

A compilation of country-level data relating to health statistics, health spending and water sanitation

By Laura Wensley

### Data Sources

The World Health Organization: 
    Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%)
    Data by country
    http://apps.who.int/gho/data/node.main.GHEDCHEGDPSHA2011
    Dataset: WHO_GDP_spend_on_Health.csv
        
The World Health Organization: 
    Basic and safely managed drinking water services 
    Data by country 
    http://apps.who.int/gho/data/node.main.WSHWATER?lang=en
    Dataset: WHO_Water_Sanitation.csv       
        
The World Health Organization: 
    Density of Medical Practitioners
    Data by country 
    http://apps.who.int/gho/data/node.main.HWFGRP_0020?lang=en
    Dataset: WHO_Medical_Practitioners.json

## Set-up

In [574]:
import pandas as pd
from sqlalchemy import create_engine
import json
import numpy as np

## Extract CSV into DataFrames

### DATA SET 1: Health Spend as a % of GDP, by country, by year - csv

In [730]:
GDP_file = "WHO_GDP_spend_on_Health.csv"
GDP_df = pd.read_csv(GDP_file,skiprows=1)
GDP_df.head()

,Country,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,Afghanistan,10.3,9.7,8.8,7.9,8.6,8.6,9.8,10.3,9.9,10.6,9.9,9.8,8.9,9.4,No data,No data
1,Albania,6.8,7.0,6.3,5.9,5.6,5.0,5.1,5.6,6.1,5.9,6.3,6.6,6.4,6.4,6.5,6.8
2,Algeria,7.1,6.5,6.0,6.0,5.3,5.1,5.4,4.2,3.8,3.4,3.2,3.5,3.6,3.7,3.8,3.5
3,Andorra,12.0,12.0,14.2,12.1,11.8,11.6,10.9,10.4,9.8,9.7,9.8,9.2,9.2,9.4,9.4,9.3
4,Angola,2.9,2.8,3.0,2.7,2.8,2.7,3.6,3.5,3.2,3.6,4.0,5.0,4.7,4.2,5.2,2.5


In [731]:
# rename columns

# GDP_df.rename(columns={GDP_df.columns[0]:"country"}) --> DID NOT WORK

headers = GDP_df.columns.tolist()
headers[0]= "country"

# headers

GDP_df.columns = headers

In [732]:
GDP_name = pd.read_csv(GDP_file,nrows=0)
title = list(GDP_name)[1]
title

'Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%)'

In [733]:
# Unpivot the dataframe from wide to long
GDP_df = pd.melt(GDP_df,id_vars=["country"], var_name="year")
GDP_df.head()

,country,year,value
0,Afghanistan,2015,10.3
1,Albania,2015,6.8
2,Algeria,2015,7.1
3,Andorra,2015,12.0
4,Angola,2015,2.9


In [734]:
# add measurement columns
GDP_df["measurement"] = title

GDP_df.head()

,country,year,value,measurement
0,Afghanistan,2015,10.3,Current health expenditure (CHE) as percentage...
1,Albania,2015,6.8,Current health expenditure (CHE) as percentage...
2,Algeria,2015,7.1,Current health expenditure (CHE) as percentage...
3,Andorra,2015,12.0,Current health expenditure (CHE) as percentage...
4,Angola,2015,2.9,Current health expenditure (CHE) as percentage...


### DATA SET 2: Water Sanitation, by Country - csv

#### DATA SCRUB: USING multi_index FUNCTION

https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.melt.html

https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

In [735]:
# open file and read csv as Dataframe without a header

water_file = "WHO_Water_Sanitation.csv"
water_df = pd.read_csv(water_file,header = None)

In [736]:
# Transpose rows 1 through 3 as a separate dataframe

index_df = water_df.iloc[:3,1:].T.rename(columns = {0:"year",1:"measurement",2:"type"})

#index_df.head()

In [737]:
# Use this new dataframe to create a MultiIndex object of levels, codes and names

multi_index = pd.MultiIndex.from_frame(index_df)

#multi_index

In [738]:
# Clean original data frams (rename columns, remove unwanted rows and set index) 

water_df2 = water_df.iloc[3:].rename(columns = {0:"country"}).set_index("country")

In [739]:
# # inspect cleaned dataframe
# water_df2.head(2)

In [740]:
# Set the columns names using the MultiIndex object
water_df2.columns = multi_index

In [741]:
# # inspect dataframe with new columns
# water_df2.head(2)

In [742]:
# reset index
water_df2 = water_df2.reset_index()

In [743]:
# # inspect dataframe with new index
# water_df2.head(2)

In [746]:
# Unpivot the dataframe from wide to long
water_df = pd.melt(water_df2,id_vars=["country"])
water_df.head()

,country,year,measurement,type,value
0,Afghanistan,2015,Population using at least basic drinking-wate...,Rural,53
1,Albania,2015,Population using at least basic drinking-wate...,Rural,90
2,Algeria,2015,Population using at least basic drinking-wate...,Rural,89
3,Andorra,2015,Population using at least basic drinking-wate...,Rural,100
4,Angola,2015,Population using at least basic drinking-wate...,Rural,23


In [747]:
# Extract years

year_list = list(set(water_df.iloc[1:,1]))
year_list.sort()
year_list

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015']

In [748]:
# Extract demographic descriptions

type_list = list(set(water_df.iloc[1:,3]))
type_list

[' Rural', ' Total', ' Urban']

In [749]:
# Extract measurement descriptions

meas_list = list(set(water_df.iloc[1:,2]))
meas_list

[' Population using safely managed drinking-water services (%)',
 ' Population using at least basic drinking-water services (%)']

### DATA SET 2: Medical Practitioners Data - JSON

https://apps.who.int/gho/athena/data/GHO/HWF_0001,HWF_0002,HWF_0003,HWF_0004,HWF_0005.json?filter=COUNTRY:*

In [901]:
# convert the jason file to a python dictionary

med_file = "WHO_Medical_Practitioners.json"
with open(med_file,"r") as f:
    who_json = json.load(f)

#### Fact Dataframe

In [902]:
# create a data frame from a dictionary from within the python dictionary

fact_df = pd.DataFrame(who_json["fact"])

fact_df.head()

,Dim,comments,dataset,effective_date,end_date,fact_id,published,value
0,"[{'category': 'PUBLISHSTATE', 'code': 'PUBLISH...",Includes Specialist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572456,True,"{'display': '293', 'numeric': 293.0, 'low': No..."
1,"[{'category': 'PUBLISHSTATE', 'code': 'PUBLISH...",Includes Generalist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572206,True,"{'display': '1114', 'numeric': 1114.0, 'low': ..."
2,"[{'category': 'PUBLISHSTATE', 'code': 'PUBLISH...",Includes Médecin spécialiste</br> Data Source:...,CYCU,2019-01-15,2900-12-31,20572170,True,"{'display': '422', 'numeric': 422.0, 'low': No..."
3,"[{'category': 'PUBLISHSTATE', 'code': 'PUBLISH...",Includes General Medical Practitioners</br> Da...,CYCU,2019-01-15,2900-12-31,20572086,True,"{'display': '17237', 'numeric': 17237.0, 'low'..."
4,"[{'category': 'PUBLISHSTATE', 'code': 'PUBLISH...",Includes Generalist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572013,True,"{'display': '7920', 'numeric': 7920.0, 'low': ..."


https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.at.html

In [903]:
# append the items in the dictionaries from the "Dim" column into columns in the "fact" dictionary

fact_df["year"] = np.nan
fact_df["published_state"] = ""
fact_df["country"] = ""
fact_df["GHO"] = ""

for index,row in fact_df.iterrows():
    dims = row["Dim"]
    for dictionary in dims:
        if dictionary["category"] == "PUBLISHSTATE":
            fact_df.at[index,"published_state"] = dictionary["code"]
        elif dictionary["category"] == "YEAR":
            fact_df.at[index,"year"] = dictionary["code"]
        elif dictionary["category"] == "COUNTRY":
            fact_df.at[index,"country"] = dictionary["code"]
        elif dictionary["category"] == "GHO":
            fact_df.at[index,"GHO"] = dictionary["code"]

fact_df.head(2)

,Dim,comments,dataset,effective_date,end_date,fact_id,published,value,year,published_state,country,GHO
0,"[{'category': 'PUBLISHSTATE', 'code': 'PUBLISH...",Includes Specialist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572456,True,"{'display': '293', 'numeric': 293.0, 'low': No...",2013.0,PUBLISHED,JAM,HWF_0004
1,"[{'category': 'PUBLISHSTATE', 'code': 'PUBLISH...",Includes Generalist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572206,True,"{'display': '1114', 'numeric': 1114.0, 'low': ...",2017.0,PUBLISHED,RWA,HWF_0003


In [904]:
# append the items in the list from the "value" column into columns in the "fact" dictionary

fact_df["display"] = np.nan
fact_df["numeric"] = np.nan
fact_df["low"] = np.nan
fact_df["high"] = np.nan
fact_df["stderr"] = np.nan
fact_df["stddev"] = np.nan

for index,row in fact_df.iterrows():
    value = row["value"]
    for x in ["display","numeric","low","high","stderr","stddev"]:
        fact_df.at[index,x] = value[x]

fact_df.head(2)

,Dim,comments,dataset,effective_date,end_date,fact_id,published,value,year,published_state,country,GHO,display,numeric,low,high,stderr,stddev
0,"[{'category': 'PUBLISHSTATE', 'code': 'PUBLISH...",Includes Specialist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572456,True,"{'display': '293', 'numeric': 293.0, 'low': No...",2013.0,PUBLISHED,JAM,HWF_0004,293.0,293.0,NaN,NaN,NaN,NaN
1,"[{'category': 'PUBLISHSTATE', 'code': 'PUBLISH...",Includes Generalist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572206,True,"{'display': '1114', 'numeric': 1114.0, 'low': ...",2017.0,PUBLISHED,RWA,HWF_0003,1114.0,1114.0,NaN,NaN,NaN,NaN


In [905]:
# Drop columns with dictionaries that we have now converted to columns within the dataframe

fact_df = fact_df.drop(["Dim","value"], axis=1)

# rename columns
fact_df = fact_df.rename(columns={"country":"country_id"})
fact_df = fact_df.rename(columns={"display":"value"})

fact_df.head(2)

,comments,dataset,effective_date,end_date,fact_id,published,year,published_state,country_id,GHO,value,numeric,low,high,stderr,stddev
0,Includes Specialist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572456,True,2013.0,PUBLISHED,JAM,HWF_0004,293.0,293.0,NaN,NaN,NaN,NaN
1,Includes Generalist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572206,True,2017.0,PUBLISHED,RWA,HWF_0003,1114.0,1114.0,NaN,NaN,NaN,NaN


#### Dimentional Dataframes

In [906]:
# create a data frame from a dictionary from within the python dictionary

dim_df = pd.DataFrame(who_json["dimension"])
dim_df

,code,display,isMeasure,label
0,"[{'label': 'HWF_0001', 'display': 'Medical doc...",Indicator,True,GHO
1,"[{'label': 'PUBLISHED', 'display': 'Published'...",PUBLISH STATES,False,PUBLISHSTATE
2,"[{'label': '1952', 'display': '1952', 'display...",Year,False,YEAR
3,"[{'label': 'AFG', 'display': 'Afghanistan', 'd...",Country,False,COUNTRY


#### Dimentional Dataframes: GHO

In [907]:
# create a data frame from a dictionary from within the python dictionary

GHO_df = pd.DataFrame(dim_df["code"][0])
GHO_df

,attr,display,display_sequence,label,url
0,"[{'category': 'CATEGORY', 'value': 'Health wor...",Medical doctors (per 10 000 population),10,HWF_0001,http://apps.who.int/gho/data/node.wrapper.imr?...
1,"[{'category': 'CATEGORY', 'value': 'Health wor...",Medical doctors (number),20,HWF_0002,http://apps.who.int/gho/data/node.wrapper.imr?...
2,"[{'category': 'CATEGORY', 'value': 'Health wor...",Generalist medical practitioners (number),30,HWF_0003,http://apps.who.int/gho/data/node.wrapper.imr?...
3,"[{'category': 'CATEGORY', 'value': 'Health wor...",Specialist medical practitioners (number),40,HWF_0004,http://apps.who.int/gho/data/node.wrapper.imr?...
4,"[{'category': 'CATEGORY', 'value': 'Health wor...",Medical doctors not further defined (number),50,HWF_0005,http://apps.who.int/gho/data/node.wrapper.imr?...


In [908]:
# attr_df = pd.DataFrame(GHO_df["attr"][0])
# attr_df

In [909]:
# append the items in the dictionary from the "attr" column into columns in the "GHO" dataframe

GHO_df["category"] = ""
GHO_df["value"] = ""

for index,row in GHO_df.iterrows():
    attribute = row["attr"]
    for x in ["category","value"]:
        GHO_df.at[index,x] = attribute[0][x]


In [910]:
# Drop columns with dictionaries that we have now converted to columns within the dataframe
GHO_df = GHO_df.drop(["attr", "category"], axis=1)

In [911]:
# rename columns
GHO_df = GHO_df.rename(columns={"display":"measurement"})
GHO_df = GHO_df.rename(columns={"label":"GHO"})
GHO_df = GHO_df.rename(columns={"value":"category"})
GHO_df

,measurement,display_sequence,GHO,url,category
0,Medical doctors (per 10 000 population),10,HWF_0001,http://apps.who.int/gho/data/node.wrapper.imr?...,Health workforce
1,Medical doctors (number),20,HWF_0002,http://apps.who.int/gho/data/node.wrapper.imr?...,Health workforce
2,Generalist medical practitioners (number),30,HWF_0003,http://apps.who.int/gho/data/node.wrapper.imr?...,Health workforce
3,Specialist medical practitioners (number),40,HWF_0004,http://apps.who.int/gho/data/node.wrapper.imr?...,Health workforce
4,Medical doctors not further defined (number),50,HWF_0005,http://apps.who.int/gho/data/node.wrapper.imr?...,Health workforce


#### Dimentional Dataframes:  PUBLISHSTATE

In [912]:
# create a data frame from a dictionary from within the python dictionary

PUBLISHSTATE_df = pd.DataFrame(dim_df["code"][1])
PUBLISHSTATE_df.head()

,attr,display,display_sequence,label,url
0,[],Published,0,PUBLISHED,


#### Dimentional Dataframes:  YEAR

In [913]:
# create a data frame from a dictionary from within the python dictionary

YEAR_df = pd.DataFrame(dim_df["code"][2])
YEAR_df.head()

,attr,display,display_sequence,label,url
0,[],1952,80478048,1952,
1,[],1953,80468047,1953,
2,[],1954,80458046,1954,
3,[],1955,80448045,1955,
4,[],1956,80438044,1956,


#### Dimentional Dataframes:  COUNTRY

In [914]:
# create a data frame from a dictionary from within the python dictionary

COUNTRY_df = pd.DataFrame(dim_df["code"][3])
COUNTRY_df.head()

,attr,display,display_sequence,label,url
0,[{'category': 'WORLD_BANK_INCOME_GROUP_GNI_REF...,Afghanistan,10,AFG,
1,[{'category': 'WORLD_BANK_INCOME_GROUP_GNI_REF...,Albania,20,ALB,
2,[{'category': 'WORLD_BANK_INCOME_GROUP_GNI_REF...,Algeria,30,DZA,
3,[{'category': 'WORLD_BANK_INCOME_GROUP_GNI_REF...,Andorra,40,AND,
4,[{'category': 'WORLD_BANK_INCOME_GROUP_GNI_REF...,Angola,50,AGO,


In [915]:
# inspect single attribute dictionary

attr_df = pd.DataFrame(COUNTRY_df["attr"][0])
#attr_df


In [916]:
# create list of unique country attributes

category_list_of_lists = []

for i in range(len(COUNTRY_df)):
    attr_df = pd.DataFrame(COUNTRY_df["attr"][i])
    category_list_0 = attr_df["category"].tolist()
    category_list_of_lists.append(category_list_0)
    
category_list = []

for x in category_list_of_lists:
    for item in x:
        category_list.append(item)

category_list = set(category_list)

category_list = list(category_list)

#category_list

In [917]:
# append the items in the dictionary from the "attr" column into columns in the "Country" dataframe

COUNTRY_df[category_list] = pd.DataFrame([["","","","","","","",np.nan,"","","","","","","","","",np.nan,np.nan,"","",""]], index=COUNTRY_df.index)


for index,row in COUNTRY_df.iterrows():
    attribute = row["attr"]
    for dictionary in attribute:
        for x in dictionary:
            for i in range(len(category_list)):
                if dictionary["category"] == category_list[i]:
                     COUNTRY_df.at[index,category_list[i]] = dictionary["value"]
                

In [918]:
# Drop columns with dictionaries that we have now converted to columns within the dataframe
COUNTRY_df = COUNTRY_df.drop(["attr"], axis=1)

In [919]:
# rename columns
COUNTRY_df = COUNTRY_df.rename(columns={"label":"country_id"})

In [920]:
# rename columns

headers = COUNTRY_df.columns.tolist()
headers[0]= "country"

# headers

COUNTRY_df.columns = headers

In [921]:
COUNTRY_df.head(2)

,country,display_sequence,country_id,url,WHO_REGION,FIPS,IOC,ISO,LANGUAGES_EN_2012,SHORTNAMEES,...,WHO,WORLD_BANK_INCOME_GROUP,GEOMETRY,LAND_AREA_KMSQ_2012,DS,WORLD_BANK_INCOME_GROUP_GNI_REFERENCE_YEAR,WORLD_BANK_INCOME_GROUP_RELEASE_DATE,WORLD_BANK_INCOME_GROUP_CODE,MARC,WMO
0,Afghanistan,10,AFG,,Eastern Mediterranean,AF,AFG,AFG,"Dari, Pashto, Turkic languages, 30 minor langu...",Afganistan,...,AFG,Low income,AFG,"652,230",AFG,2017.0,2018.0,WB_LI,af,AF
1,Albania,20,ALB,,Europe,AL,ALB,ALB,"Albanian, Greek, Vlach, Romani, Slavic dialects",Albania,...,ALB,Upper middle income,ALB,"27,400",AL,2017.0,2018.0,WB_UMI,aa,AB


#### Join Dataframes

In [925]:
# Combine the data into a single dataset
med_data= pd.merge(fact_df, GHO_df, on="GHO", how="left")
med_data= pd.merge(med_data, COUNTRY_df, on="country_id", how="left")
med_data.head()

,comments,dataset,effective_date,end_date,fact_id,published,year,published_state,country_id,GHO,...,WHO,WORLD_BANK_INCOME_GROUP,GEOMETRY,LAND_AREA_KMSQ_2012,DS,WORLD_BANK_INCOME_GROUP_GNI_REFERENCE_YEAR,WORLD_BANK_INCOME_GROUP_RELEASE_DATE,WORLD_BANK_INCOME_GROUP_CODE,MARC,WMO
0,Includes Specialist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572456,True,2013.0,PUBLISHED,JAM,HWF_0004,...,JAM,Upper middle income,JAM,"10,830",JA,2017.0,2018.0,WB_UMI,jm,JM
1,Includes Generalist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572206,True,2017.0,PUBLISHED,RWA,HWF_0003,...,RWA,Low income,RWA,"24,670",RWA,2017.0,2018.0,WB_LI,rw,RW
2,Includes Médecin spécialiste</br> Data Source:...,CYCU,2019-01-15,2900-12-31,20572170,True,2011.0,PUBLISHED,CMR,HWF_0004,...,CAE,Lower middle income,CMR,"472,710",CAM,2017.0,2018.0,WB_LMI,cm,CM
3,Includes General Medical Practitioners</br> Da...,CYCU,2019-01-15,2900-12-31,20572086,True,2013.0,PUBLISHED,LKA,HWF_0003,...,SRL,Lower middle income,LKA,"62,710",CL,2017.0,2018.0,WB_LMI,ce,SB
4,Includes Generalist Medical Practitioners </br...,CYCU,2019-01-15,2900-12-31,20572013,True,2016.0,PUBLISHED,SGP,HWF_0003,...,SIN,High income,SGP,700,SGP,2017.0,2018.0,WB_HI,si,SR


In [926]:
med_data.columns

Index(['comments', 'dataset', 'effective_date', 'end_date', 'fact_id',
       'published', 'year', 'published_state', 'country_id', 'GHO', 'value',
       'numeric', 'low', 'high', 'stderr', 'stddev', 'measurement',
       'display_sequence_x', 'url_x', 'category', 'country',
       'display_sequence_y', 'url_y', 'WHO_REGION', 'FIPS', 'IOC', 'ISO',
       'LANGUAGES_EN_2012', 'SHORTNAMEES', 'WHOLEGALSTATUS', 'MORT',
       'SHORTNAMEFR', 'ITU', 'WHO_REGION_CODE', 'ISO2', 'WHO',
       'WORLD_BANK_INCOME_GROUP', 'GEOMETRY', 'LAND_AREA_KMSQ_2012', 'DS',
       'WORLD_BANK_INCOME_GROUP_GNI_REFERENCE_YEAR',
       'WORLD_BANK_INCOME_GROUP_RELEASE_DATE', 'WORLD_BANK_INCOME_GROUP_CODE',
       'MARC', 'WMO'],
      dtype='object')

In [927]:
# Drop unwanted columns
med_data = med_data.drop(['comments','dataset', 'effective_date', 'end_date', 'fact_id',
       'published', 'published_state', 'country_id','GHO',
       'numeric', 'low', 'high', 'stderr', 'stddev',
       'display_sequence_x', 'url_x', 'category', 'display_sequence_y', 'url_y', 
       'FIPS', 'IOC', 'ISO',
       'LANGUAGES_EN_2012', 'SHORTNAMEES', 'WHOLEGALSTATUS', 'MORT',
       'SHORTNAMEFR', 'ITU', 'WHO_REGION_CODE', 'ISO2', 'WHO',
       'GEOMETRY', 'DS',
       'WORLD_BANK_INCOME_GROUP_GNI_REFERENCE_YEAR',
       'WORLD_BANK_INCOME_GROUP_RELEASE_DATE', 'WORLD_BANK_INCOME_GROUP_CODE',
       'MARC', 'WMO'], axis=1)

In [928]:
med_data

,year,value,measurement,country,WHO_REGION,WORLD_BANK_INCOME_GROUP,LAND_AREA_KMSQ_2012
0,2013.0,293.000,Specialist medical practitioners (number),Jamaica,Americas,Upper middle income,"10,830"
1,2017.0,1114.000,Generalist medical practitioners (number),Rwanda,Africa,Low income,"24,670"
2,2011.0,422.000,Specialist medical practitioners (number),Cameroon,Africa,Lower middle income,"472,710"
3,2013.0,17237.000,Generalist medical practitioners (number),Sri Lanka,South-East Asia,Lower middle income,"62,710"
4,2016.0,7920.000,Generalist medical practitioners (number),Singapore,Western Pacific,High income,700
5,2016.0,20000.000,Generalist medical practitioners (number),Sri Lanka,South-East Asia,Lower middle income,"62,710"
6,2009.0,114.000,Specialist medical practitioners (number),Suriname,Americas,Upper middle income,"156,000"
7,2012.0,280.000,Specialist medical practitioners (number),Jamaica,Americas,Upper middle income,"10,830"
8,2010.0,4880.000,Generalist medical practitioners (number),Lao People's Democratic Republic,Western Pacific,Lower middle income,"230,800"
9,2010.0,54273.000,Specialist medical practitioners (number),Mexico,Americas,Upper middle income,"1,943,950"


# Test